In [1]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml

--2024-05-12 09:16:28--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74162295 (71M) [text/plain]
Saving to: ‘posts_sample.xml’

posts_sample.xml    100%[===================>]  70.73M   336KB/s    in 5m 28s  

2024-05-12 09:21:57 (221 KB/s) - ‘posts_sample.xml’ saved [74162295/74162295]



In [2]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

--2024-05-12 09:21:57--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40269 (39K) [text/plain]
Saving to: ‘programming-languages.csv’

programming-languag 100%[===================>]  39.33K   129KB/s    in 0.3s    

2024-05-12 09:22:01 (129 KB/s) - ‘programming-languages.csv’ saved [40269/40269]



In [3]:
!pip install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 16.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=234bcde057cf7eacdfcbbf2691acc67d1b0c6b5572c85279c2f65e859e6d0141
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [4]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [5]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("L2 - Reports with Apache Spark").master("local[*]").getOrCreate()
sc

In [6]:
prog_lang = sc.read.csv("programming-languages.csv")
prog_lang_list = [str(x[0]) for x in prog_lang.collect()]
prog_lang_list[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

In [7]:
posts = sc.read.format("xml").options(rowTag="row").load('posts_sample.xml')
posts.take(1)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

In [8]:
def language_detection(x):
  tag = None
  for language in prog_lang_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)


def check_date(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end

In [9]:
final_result = {}
for year in range(2010, 2020):
  final_result[year] = posts.rdd\
      .filter(lambda x: x._Tags is not None and check_date(x, year))\
      .map(language_detection)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,)\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()
  final_result[year] = final_result[year].select(col("_1").alias("Programming_language"),
                                                 col("_2").alias(f"Number_of_mentions_in_{year}"))\
                                         .limit(10)
  final_result[year].show()

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2010|
+--------------------+--------------------------+
|                Java|                        52|
|          JavaScript|                        44|
|                 PHP|                        42|
|              Python|                        25|
|         Objective-C|                        22|
|                   C|                        20|
|                Ruby|                        11|
|              Delphi|                         7|
|                   R|                         3|
|                Bash|                         3|
+--------------------+--------------------------+

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2011|
+--------------------+--------------------------+
|                 PHP|                        97|
|                Java|                        92|
|          JavaScript|                        82|

In [ ]:
for year in final_result.keys():
    final_result[year].write.format("parquet").save(f"top_{year}")